In [1]:
import MetaTrader5 as mt5

In [2]:
from trader_models import SRUTrader, GPT2Config

import torch
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import pytz

stds = [0.00020527, 0.00015178, 0.00015272, 0.00022341, 0.00463119]

In [3]:
mt5.initialize(path = "C:/Program Files/Coinexx MT5 Terminal/terminal64.exe",
               login = 806219,
               server = "Coinexx-Demo",
               password = "@Ma4J@p0#@epM#0")

True

In [182]:
config = GPT2Config(
    n_embd = 448, n_head = 7
)
model = SRUTrader.from_pretrained('srupp.model', config = config).cuda().eval()

In [241]:
import time
from IPython.display import clear_output

while True:
    clear_output(wait = True)
    # mt5 assumes you'll pass a local timezone
    bars = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M1, datetime.now(), 1440)
    bars = pd.DataFrame(bars)

    # use coinexx timezone to avoid confusion (where 0:00 is start of trading day)
    bars['time'] = pd.to_datetime(bars['time'], unit='s') + timedelta(hours = 3)
    bars = bars.set_index('time')

    # preprocessing (same code as original)
    bars['day_close'] = bars['close'].sub(bars.groupby(bars.index.day)['close'].transform('first'), axis = 0)
    bars[['open', 'high', 'low']] = bars[['open', 'high', 'low']].sub(bars['close'], axis = 0)
    bars['close'] = bars['close'] - bars['close'].shift(1)

    # filter to only today
    bars = bars[bars.index.day == bars.index.day[-1]]

    bars = bars[['open', 'high', 'low', 'close', 'day_close']]
    bars = bars.div(stds['EURUSD'], axis = 1)

    # # get rid of partial last close
    # bars = bars.iloc[:-1]
    display(bars)

    with torch.no_grad():
        model = model.eval()
        display((model(torch.Tensor(bars.values).unsqueeze(0).cuda())[0][-1] * 100).round())
        display((model(torch.Tensor(bars.values)[:-1].unsqueeze(0).cuda())[0][-1] * 100).round())
    
    time.sleep(5)

,open,high,low,close,day_close
time,,,,,
2022-10-27 00:00:00,-0.828178,0.197654,-1.113148,0.760933,0.000000
2022-10-27 00:01:00,-0.292298,0.395309,-0.458355,0.223804,0.010796
2022-10-27 00:02:00,0.438447,0.658848,-0.065479,-0.313325,-0.004319
2022-10-27 00:03:00,0.438447,1.120042,-0.065479,-0.402847,-0.023752
2022-10-27 00:04:00,-0.243582,0.263539,-0.720272,0.223804,-0.012956
...,...,...,...,...,...
2022-10-27 05:54:00,0.097433,0.592963,-0.065479,-0.089522,0.233201
2022-10-27 05:55:00,-0.438447,0.000000,-0.654793,0.402847,0.252635
2022-10-27 05:56:00,0.389731,0.724733,-0.196438,-0.402847,0.233201


tensor([ 1.,  1.,  0., -1., -1., -0., -0.,  0.,  1.,  1., -2., -1., -2., -1.,
        -1., -0., -2., -1., -1., -1., -1., -1., -0., -1., -1., -2., -0., -1.,
        -2., -1., -2., -0., -2., -2., -1., -2., -1., -1., -2., -1., -1., -2.,
        -1., -1., -2., -1., -1., -2., -1., -0., -2., -2., -0., -2.,  0.,  1.,
        -2., -2., -1., -1.], device='cuda:0')

tensor([ 0.,  0., -0., -1., -2., -1., -1., -0.,  0., -0., -2., -1., -3., -2.,
        -2., -1., -3., -1., -2., -1., -2., -2., -1., -2., -1., -3., -1., -1.,
        -3., -2., -3., -1., -2., -2., -2., -2., -1., -2., -2., -1., -1., -2.,
        -1., -3., -2., -2., -1., -2., -1., -1., -3., -1., -1., -1., -1.,  0.,
        -2., -1., -1., -2.], device='cuda:0')

KeyboardInterrupt: 

In [246]:
with torch.no_grad():
    display((model(torch.Tensor(bars.values).unsqueeze(0).cuda())[0].mean(dim = 1)* 100).round())

tensor([-6., -7.,  7.,  9.,  5., -2.,  1., -3., -6., -3., -3., -6., -5., -1.,
         2.,  6.,  1.,  6.,  2.,  4.,  2.,  6.,  6.,  8.,  6.,  4.,  2.,  6.,
         2.,  4.,  6.,  1.,  4.,  2.,  0., -0.,  1., -2.,  0.,  1.,  4.,  6.,
         5.,  1.,  5.,  1.,  1.,  4.,  6.,  4.,  6.,  6.,  8.,  7.,  8.,  9.,
         9.,  5.,  2., -0.,  5.,  3.,  4.,  5.,  7.,  4.,  5.,  6.,  5.,  1.,
         5.,  6.,  7.,  5.,  7.,  6.,  7.,  8.,  4.,  2.,  5.,  5.,  6.,  6.,
         2.,  1.,  4.,  5.,  4.,  6.,  3.,  5.,  1., -2.,  4.,  2.,  2.,  3.,
         5.,  3.,  1.,  1., -1.,  2.,  4.,  2.,  4.,  4.,  3., -0.,  4.,  5.,
         2., -1., -3., -3., -2., -2.,  2., -2.,  3.,  3.,  1., -1., -3., -3.,
        -5., -1., -0.,  1.,  3.,  1., -1., -2.,  2.,  1.,  2.,  5.,  1.,  4.,
         6.,  3.,  4.,  0.,  4.,  2.,  6.,  2.,  1.,  4.,  2.,  1.,  1.,  4.,
         2., -1.,  0., -3., -5., -6.,  0., -0., -3., -5., -4., -4., -1., -2.,
        -3., -1., -0., -3.,  1., -2.,  1., -1.,  2., -1.,  2.,  